In [53]:
df_ibge = pd.read_csv("C:\\Users\\crist\\OneDrive\\Documentos\\00_CASA\\CRISTIANO\\EDUCAÇÃO\\16_FIAP\\geodata-br\\municipiosBR.csv", delimiter=',')
# df_ibge = pd.read_csv("C:\\Users\\crist\\OneDrive\\Documentos\\00_CASA\\CRISTIANO\\EDUCAÇÃO\\16_FIAP\\geodata-br\\municipios.csv",  encoding='latin-1', delimiter=';')

# Renomear a coluna 'CÓDIGO DO MUNICÍPIO - IBGE' para 'id' para facilitar o merge
df_ibge.rename(columns={'municipio': 'id'}, inplace=True)

# # Renomear a coluna 'MUNICÍPIO - IBGE' para 'nome_municipio' para facilitar o merge
# df_ibge.rename(columns={'MUNICÍPIO - IBGE': 'nome_municipio'}, inplace=True)

# # Renomear a coluna 'UF' para 'uf' para facilitar o merge
# df_ibge.rename(columns={'UF': 'uf'}, inplace=True)

# # Selecionar apenas as colunas necessárias
# df_ibge = df_ibge[['id', 'nome_municipio', 'uf']]

# Cast da coluna 'id' para string para garantir a compatibilidade no merge
df_ibge['id'] = df_ibge['id'].astype(str)

df_ibge.head()
# df.info()


,id,uf,uf_code,name,mesoregion,microregion,rgint,rgi,osm_relation_id,wikidata_id,is_capital,wikipedia_pt,lon,lat,no_accents,slug_name,alternative_names,pop_21
0,1100015,11,RO,Alta Floresta d'Oeste,1102,11006,1102,110005,325842,Q1761456,NaN,Alta Floresta d'Oeste,-61.999824,-11.935540,Alta Floresta d'Oeste,alta-floresta-doeste,NaN,22516
1,1100023,11,RO,Ariquemes,1102,11003,1101,110002,325888,Q1646975,NaN,Ariquemes,-63.033269,-9.908463,Ariquemes,ariquemes,NaN,111148
2,1100031,11,RO,Cabixi,1102,11008,1102,110006,325882,Q940433,NaN,Cabixi,-60.544314,-13.499763,Cabixi,cabixi,NaN,5067
3,1100049,11,RO,Cacoal,1102,11006,1102,110005,325872,Q1761637,NaN,Cacoal,-61.442944,-11.433865,Cacoal,cacoal,NaN,86416
4,1100056,11,RO,Cerejeiras,1102,11008,1102,110006,325883,Q1803210,NaN,Cerejeiras,-60.818426,-13.195033,Cerejeiras,cerejeiras,NaN,16088


In [54]:
import pandas as pd
import json
import os

# 1. Defina o caminho para a pasta com os arquivos GeoJSON
# (O 'r' antes da string é importante para tratar as barras invertidas do Windows)
target_directory = r"C:\Users\crist\OneDrive\Documentos\00_CASA\CRISTIANO\EDUCAÇÃO\16_FIAP\geodata-br\geojson"

# 2. Lista para armazenar os dados de TODOS os arquivos
all_data_list = []

print(f"Iniciando processamento da pasta: {target_directory}\n")

try:
    # 3. Listar todos os arquivos no diretório
    for file_name in os.listdir(target_directory):
        
        # 4. Verificar se o arquivo é um .json
        if file_name.endswith('.json'):
            file_path = os.path.join(target_directory, file_name)
            print(f"Processando arquivo: {file_name}...")
            
            try:
                # 5. Abrir e carregar o arquivo JSON
                with open(file_path, 'r', encoding='utf-8') as f:
                    geojson_data = json.load(f)

                # 6. Processar o conteúdo do arquivo
                if geojson_data.get('type') == 'FeatureCollection' and 'features' in geojson_data:
                    for feature in geojson_data['features']:
                        properties = feature.get('properties', {})
                        feature_id = properties.get('id')
                        feature_name = properties.get('name')
                        
                        geometry = feature.get('geometry', {})
                        geometry_str = json.dumps(geometry)

                        # 7. Adicionar à lista consolidada
                        all_data_list.append({
                            'id': feature_id,
                            'name': feature_name,
                            'geoJson': geometry_str
                        })
                else:
                    print(f"  Aviso: O arquivo {file_name} não parece ser um FeatureCollection válido.")
            
            except json.JSONDecodeError:
                print(f"  Erro: Falha ao decodificar JSON no arquivo {file_name}.")
            except Exception as e:
                print(f"  Erro inesperado ao processar {file_name}: {e}")

    # 8. Criar o DataFrame consolidado
    if all_data_list:
        df = pd.DataFrame(all_data_list)

        # Selecionar apenas colunas relevantes
        df = df[['id', 'geoJson']]

        # Remover duplicatas, se houver
        df.drop_duplicates(inplace=True)
        
        print("\n--- Processamento Concluído ---")
        print("DataFrame consolidado criado com sucesso:")
        print(df.head())
        print(f"\nTotal de registros extraídos: {len(df)}")
        df.info()

        # 10. Realizar o join com o dataset do IBGE
        df_enriched = pd.merge(df, df_ibge, on='id', how='left')
        print(df_enriched.head())

        # Salvar o dataset enriquecido em um novo arquivo CSV
        output_csv_name = 'dataset_enriquecido_geo.csv'
        df_enriched.to_csv(output_csv_name, index=False)

        print(f"\nDataset salvo com sucesso no arquivo: {output_csv_name}")
    else:
        print("\nNenhum dado foi extraído. Verifique se a pasta contém arquivos .json válidos.")

except FileNotFoundError:
    print(f"Erro: O diretório não foi encontrado: {target_directory}")
except Exception as e:
    print(f"Ocorreu um erro inesperado ao acessar a pasta: {e}")

Iniciando processamento da pasta: C:\Users\crist\OneDrive\Documentos\00_CASA\CRISTIANO\EDUCAÇÃO\16_FIAP\geodata-br\geojson

Processando arquivo: geojs-100-mun.json...
Processando arquivo: geojs-11-mun.json...
Processando arquivo: geojs-12-mun.json...
Processando arquivo: geojs-13-mun.json...
Processando arquivo: geojs-14-mun.json...
Processando arquivo: geojs-15-mun.json...
Processando arquivo: geojs-16-mun.json...
Processando arquivo: geojs-17-mun.json...
Processando arquivo: geojs-21-mun.json...
Processando arquivo: geojs-22-mun.json...
Processando arquivo: geojs-23-mun.json...
Processando arquivo: geojs-24-mun.json...
Processando arquivo: geojs-25-mun.json...
Processando arquivo: geojs-26-mun.json...
Processando arquivo: geojs-27-mun.json...
Processando arquivo: geojs-28-mun.json...
Processando arquivo: geojs-29-mun.json...
Processando arquivo: geojs-31-mun.json...
Processando arquivo: geojs-32-mun.json...
Processando arquivo: geojs-33-mun.json...
Processando arquivo: geojs-35-mun.j